In [1]:

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import seaborn as sns
import matplotlib.pyplot as plt

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

In [2]:
class BaseAgent(Agent):
    def __init__(self, model, stamina):
        super().__init__(model)
        self.stamina = stamina

    def move(self):
        """Move to a random adjacent cell."""
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
    
    def rest(self):
        """Rest to regain stamina."""
        if self.stamina == 0:
            self.stamina += 10    

In [3]:
class Miner(BaseAgent):
    def __init__(self, model, stamina=69):
        super().__init__(model, stamina)

    def step(self):
        if self.stamina > 0:
            if self.near_drill():
                self.collect_or_refuel()
            elif self.near_lifepod():
                self.deliver_resources()
            else:
                self.move()
            self.stamina -= 1
        else:
            self.rest()

    def near_drill(self):
        """Check if the miner is near the drill."""
        return any(isinstance(agent, Drill) for agent in self.model.grid.get_cell_list_contents([self.pos]))

    def near_lifepod(self):
        """Check if the miner is near the lifepod."""
        return self.pos == self.model.lifepod_location

    def collect_or_refuel(self):
        """Collect iron or refuel the drill."""
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Drill):
                if agent.iron > 0:
                    agent.iron -= 3  # Collect iron
                elif agent.fuel < 1:
                    agent.fuel += 20 # Refuel

    def deliver_resources(self):
        """Deliver resources to the lifepod."""
        # Find the lifepod in the current cell
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Lifepod):
                # Transfer iron to the lifepod
                agent.iron += 3
                break

       



In [4]:
class Engineer(BaseAgent):
    def __init__(self, model, stamina =69,):
        super().__init__( model, stamina)

    def step(self):
        if self.stamina > 0:
            if self.near_broken_drill():
                self.repair()
            else:
                self.move()
            self.stamina -= 1
        else:
            self.rest()

    def near_broken_drill(self):
        """Check if the engineer is near a broken drill."""
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        return any(isinstance(agent, Drill) and agent.is_broken() for agent in cell_contents)

    def repair(self):
        """Repair the drill."""
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Drill) and agent.is_broken():
                agent.repair()
                self.stamina -= 10  # Repairing costs stamina


In [5]:
class Drill(Agent):
    def __init__(self, model, max_fuel=69, health = 69):
        super().__init__( model)
        self.fuel = max_fuel
        self.iron = 0
        self.max_fuel = max_fuel
        self.health = health

    def is_broken(self):
        """Check if the drill is broken."""
        if self.health == 0:
          return True

    def repair(self):
        """Repair the drill."""
        self.health = 69

    def step(self):
        if not self.is_broken() and self.fuel > 2:
            self.fuel -=3
            self.iron += 3
            # need some reason to deplete health as well

In [12]:
class Farmer(BaseAgent):
    def __init__(self, model, stamina=69):
        super().__init__(model, stamina)

    def step(self):
        if self.stamina > 0:
            if self.near_greenhouse():
                self.collect_food()
            elif self.near_lifepod():
                self.deliver_food()
            else:
                self.move()
            self.stamina -= 1
        else:
            self.rest()

    def near_greenhouse(self):
        """Check if the farmer is near the greenhouse."""
        return any(isinstance(agent, Greenhouse) for agent in self.model.grid.get_cell_list_contents([self.pos]))

    def near_lifepod(self):
        """Check if the farmer is near the lifepod."""
        return self.pos == self.model.lifepod_location

    def collect_food(self):
        """Collect food from the greenhouse."""
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Greenhouse):
                if agent.food > 0:
                    agent.food -= 5  # Collect food

    def deliver_food(self):
        """Deliver resources to the lifepod."""
        # Find the lifepod in the current cell
        cell_contents = self.model.grid.get_cell_list_contents([self.pos])
        for agent in cell_contents:
            if isinstance(agent, Lifepod):
                # Transfer iron to the lifepod
                agent.food += 5
                break
      


In [7]:
class Greenhouse(Agent):
    def __init__(self, model, max_food=50):
        super().__init__(model)
        self.food = 0
        self.max_food = max_food

    def step(self):
        if self.food <= self.max_food:
            self.food += 5


In [8]:
class Lifepod(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.food = 20
        self.iron = 0
#for the sake of simplicity there's no storage cap 

In [9]:
class SurvivalModel(Model):
    """The goddamn model """
    
    def __init__(self, width, height, seed=69):
        super().__init__(seed=seed)
        self.grid = MultiGrid(width, height, True)
        
        # Set lifepod location at the center of the grid
        self.lifepod_location = (width // 2, height // 2)
        
        # Create and place the Lifepod
        self.lifepod = Lifepod(self)
        self.grid.place_agent(self.lifepod, self.lifepod_location)
        
        # Create and place worker agents at the lifepod location
        self.miner = Miner(self)
        self.engineer = Engineer(self)
        self.farmer = Farmer(self)
        
        for agent in [self.miner, self.engineer, self.farmer]:
            self.grid.place_agent(agent, self.lifepod_location)
        
        # Create and place resource-generating structures at random locations
        self.drill = Drill(self)
        self.greenhouse = Greenhouse(self)
        
        for structure in [self.drill, self.greenhouse]:
            x = self.rng.integers(0, self.grid.width)
            y = self.rng.integers(0, self.grid.height)
            while (x, y) == self.lifepod_location:  # Ensure structures aren't placed at lifepod
                x = self.rng.integers(0, self.grid.width)
                y = self.rng.integers(0, self.grid.height)
            self.grid.place_agent(structure, (x, y))
        
        # Create DataCollector to track model state
        self.datacollector = DataCollector(
            model_reporters={
                "Food": lambda m: m.lifepod.food,
                "Iron": lambda m: m.lifepod.iron,
                "Drill Health": lambda m: m.drill.health,
                "Drill Fuel": lambda m: m.drill.fuel,
                "Greenhouse Food": lambda m: m.greenhouse.food,
                "Cell Contents": self.record_cell_contents
            }
        )
    
    def record_cell_contents(self):
        """Record the contents of each cell in the grid."""
        cell_data = {}
        for cell_contents, (x, y) in self.grid.coord_iter():
            cell_data[(x, y)] = [type(agent).__name__ for agent in cell_contents]
        return cell_data
    
    def step(self):
        """Advance the model by one step."""
        self.datacollector.collect(self)
        
        self.agents.shuffle_do("step")

In [13]:
model = SurvivalModel(10, 10)
for _ in range(30):  
    model.step()

In [14]:
model_data = model.datacollector.get_model_vars_dataframe() 
print(model_data)

    Food  Iron  Drill Health  Drill Fuel  Greenhouse Food  \
0     20     0            69          69                0   
1     25     3            69          66                5   
2     30     6            69          63               10   
3     35     9            69          60               15   
4     40    12            69          57               20   
5     45    15            69          54               25   
6     50    18            69          51               30   
7     55    21            69          48               35   
8     60    24            69          45               40   
9     65    27            69          42               45   
10    70    30            69          39               50   
11    75    33            69          36               55   
12    80    36            69          33               55   
13    85    39            69          30               55   
14    90    42            69          27               55   
15    95    45          